In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Práctica de selección de características

## Objetivos

El objetivo de esta práctica es trabajar con algunas de las técnicas de selección de características estudiadas en  las sesiones teóricas utilizando scikit learn. Debes aplicar los conocimientos adquiridos en las clases de teoría a la resolución de los casos que se proponen y analizar los resultados.

## Enunciado
En esta práctica trabajaremos la selección de características mediante filtros. Para ello, se presenta un ejemplo completo de selección de características en el caso de predictores categóricos y respuesta categórica. El ejemplo incluye:

* Lectura de un dataset
* Codificación del dataset
* Selección de características usando $\chi^{2}$
* Selección de características usando *mutual information*
* Comparación de ambas selecciones usando un modelo de test

La tarea que has de realizar consiste en completar los dos casos propuestos en este block de notas de manera análoga al ejemplo presentado.

Para ello será necesario estudiar y comprender el caso resuelto y usar la documentación de [scikitlearn](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection) para resolver los dos casos pendientes:
* Predictores numéricos, respuesta categórica
* Predictores numéricos, respuesta numérica

El entregable de esta tarea consistirá en un block de notas, con los siguientes requisitos, completado por el grupo:

* **Código** para resolver los casos propuestos
* **Explicación del código**: comenta tu código de forma que se entienda cómo funciona.
* **Análisis e interpretación de los resultados**: analiza y comenta los resultados y relaciónalos con la teoría. Esto incluye el análisis e interpretación de los resultados del caso resuelto. Emplea gráficas para ilustrar el análisis cuando lo consideres oportuno.

Has de entregar un fichero comprimido con **ZIP** (y sólo ZIP) que contenga lo siguiente:
* **Block de notas** con el código, los análisis y los comentarios de *todos* los casos (incluido el que está resuelto)
* **Directorio con los datos**. Los nombres de ficheros que uses en el block de notas **han de ser relativos** a ese directorio (han de empezar por `'./nombre_directorio'`; ver lectura de fichero del caso resuelto). La idea es que al descomprimir el fichero ZIP se pueda ejecutar el block de notas sin tener que cambiar ni una coma.
* **Este trabajo se hace en grupo**, así que basta con que un integrante del grupo haga la entrega.

# Filtros

## Caso resuelto: predictores categóricos, respuesta categórica

dataset: `breast-cancer`

### Lectura y codificación del dataset

In [ ]:
# example of mutual information feature selection for categorical data
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from matplotlib import pyplot

# load the dataset
def load_dataset(filename):
    # load the dataset as a pandas DataFrame
    data = read_csv(filename, header=None)
    # retrieve numpy array
    dataset = data.values
    # split into input (X) and output (y) variables
    X = dataset[:, :-1]
    y = dataset[:,-1]
    # format all fields as string
    X = X.astype(str)
    return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc


# load the dataset
X, y = load_dataset('./datos/'+'breast-cancer.csv')
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

In [ ]:
column_names = ['age', 'menopause', 'tumor-size', 'inv-nodes', 
                'node-caps','deg-malig', 'breast', 'breast-quad','Class']
pd.DataFrame(X_train, columns=column_names)

In [ ]:
pd.DataFrame(X_train_enc, columns=column_names)

### Mutual information

In [ ]:
# feature selection mutual information
def select_features(X_train, y_train, X_test):
    fs = SelectKBest(score_func=mutual_info_classif, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc)
# what are scores for the features
print(sorted(fs.scores_))
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

### $\chi^{2}$     (chi cuadrado)

In [ ]:
# feature selection
from sklearn.feature_selection import chi2

def select_features(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k=4)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train_enc, X_test_enc)
print(X_train_fs.shape)
# what are scores for the features
for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

¿Cómo saber qué características se han seleccionado?

In [ ]:
print(fs.get_feature_names_out())
print(fs.get_support())
print(np.nonzero(fs.get_support()))
print([column_names[idx] for (idx, item) in enumerate(fs.get_support()) if item])

### Comparación de rendimiento

Evaluación de las dos selecciones de características usando como referencia un modelo de regresión logística

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def mi_mutual_info_classif(*args):
    score = mutual_info_classif(args[0], args[1], 
                                discrete_features=True)
    return score

def select_features(X_train, y_train, X_test, func, k=4):
    fs = SelectKBest(score_func=func, k=k)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

l = []
model = LogisticRegression(solver='lbfgs')

for k in range(1, 10):
#     print(f'k={k}')

    # feature selection
    X_train_fs, X_test_fs, fs = select_features(X_train_enc, 
                                                y_train_enc, 
                                                X_test_enc, chi2, k)
    # fit the model using MI
    model.fit(X_train_fs, y_train_enc)
    # evaluate the model
    yhat = model.predict(X_test_fs)
    # evaluate predictions
    accuracy = accuracy_score(y_test_enc, yhat)
#     print('Accuracy chi2: %.2f' % (accuracy*100))

    # feature selection
    X_train_fs, X_test_fs, fs = select_features(X_train_enc, 
                                                y_train_enc, 
                                                X_test_enc,
#                                                 mutual_info_classif,
                                                mi_mutual_info_classif,
                                                k)

    # fit the model using mutual information
    model.fit(X_train_fs, y_train_enc)
    # evaluate the model
    yhat = model.predict(X_test_fs)
    # evaluate predictions
    accuracy_mi = accuracy_score(y_test_enc, yhat)
#     print('Accuracy mut. inf.: %.2f' % (accuracy_mi*100))
#     print('')
    l += [(k, accuracy, accuracy_mi)]

In [ ]:
df = pd.DataFrame({'chi2': [k for _, k, _ in l],
                   'mi': [k for _, _, k in l]}, index=range(1,10))
from IPython.display import display
display(df)

df.plot(style='o-', grid=True, figsize=(9,6),
        xticks=[i for i in range(1,10)],
        title='Precisión vs. número de características', 
        xlabel='Número de características',
        ylabel='accuracy score')
plt.show()

## Caso propuesto 1: predictores numéricos, respuesta categórica

dataset: `pima-indians-diabetes`

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif, mutual_info_classif, chi2

### Lectura del dataset

In [ ]:
headers = [
    'npreg',
    'glucose',
    'diastolic',
    'triceps',
    'insulin',
    'bmi',
    'dpf',
    'age',
    'class'
]

df = pd.read_csv('./datos/pima-indians-diabetes.csv', names=headers)
df.head()

In [ ]:
X, y = df.drop('class', axis=1), df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape

### Estadístico F

In [ ]:
fs_fstat = SelectKBest(score_func=f_classif, k=4).fit(X_train, y_train)
X_train_fs = fs.transform(X_train)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.bar_label(plt.gca().containers[0], fmt='%1.2f')
plt.xticks([i for i in range(len(fs.scores_))], X_train.columns)
plt.show()

### Mutual information (caso clasificación)

In [ ]:
fs_mi = SelectKBest(score_func=mutual_info_classif, k=4).fit(X_train, y_train)
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
plt.bar_label(plt.gca().containers[0], fmt='%1.2f')
plt.xticks([i for i in range(len(fs.scores_))], X_train.columns)
plt.show()

### Modelo con todas las características

Usar el modelo `LogisticRegression(solver='liblinear')` y la métrica `accuracy_score`

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(solver='liblinear').fit(X_train, y_train)
yhat = model.predict(X_test)
accuracy_score(y_test, yhat)

### Modelo con las 4 mejores características descubiertas con F

In [ ]:
X_train_fsf = fs_fstat.transform(X_train)
X_test_fsf  = fs_fstat.transform(X_test)

model = LogisticRegression(solver='liblinear').fit(X_train_fsf, y_train)
yhat = model.predict(X_test_fsf)
accuracy_score(y_test, yhat)

### Modelo con las 4 mejores características descubiertas con *mutual information*

In [ ]:
X_train_fsmi = fs_mi.transform(X_train)
X_test_fsmi  = fs_mi.transform(X_test)

model = LogisticRegression(solver='liblinear').fit(X_train_fsmi, y_train)
yhat = model.predict(X_test_fsmi)
accuracy_score(y_test, yhat)

### Comparativa de resultados  y comentarios

## Caso propuesto 2: predictores numéricos, respuesta numérica

En este caso usaremos un dataset sintético, generado con la función `make_regression()` de scikit learn

### Generación del dataset sintético

In [ ]:
from sklearn.datasets import make_regression

# generate regression dataset
X, y = make_regression(n_samples=1000, n_features=100, n_informative=10,
                       noise=0.1, random_state=1)
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                    random_state=1)
# summarize
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

### Regresion (`f_regression()`)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# feature selection
fs = SelectKBest(score_func=f_regression, k=10).fit(X_train, y_train)
fs.scores_

### Mutual information (caso regresión)

### Modelo con todas las características

Usar el modelo `LinearRegression()` y la métrica `mean_absolute_error`

### Modelo con las 10 mejores características obtenidas con la regresión

### Modelo con las 10 mejores características obtenidas con *mutual information*

### Comparativa de resultados y comentarios